<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задание" data-toc-modified-id="Задание-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задание</a></span></li><li><span><a href="#Задание-*" data-toc-modified-id="Задание-*-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Задание *</a></span><ul class="toc-item"><li><span><a href="#Список-объектов" data-toc-modified-id="Список-объектов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Список объектов</a></span></li></ul></li><li><span><a href="#Данные-по-определенному-объекту" data-toc-modified-id="Данные-по-определенному-объекту-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Данные по определенному объекту</a></span></li></ul></div>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
import requests
import pandas as pd
from tqdm import tqdm
import sqlite3
import os
import wget

## Список объектов

In [45]:
# Для скачивания данных по

limit_ = 500
all_objects = []
for i in tqdm(range(66)):
    offset_ = limit_ * i
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc'
    res = requests.get(url)
    objects_data = res.json()
    all_objects += objects_data.get('data').get('list')

100%|██████████| 66/66 [06:57<00:00,  6.33s/it]


In [46]:
objids = [i.get('objId') for i in all_objects]

In [49]:
"""

Проверяем общее количество выгруженных объектов, смотрим сколько из них уникальные.
Общее число объектов соответствует числу с сайта, поэтому, можно сделать вывод, что некоторые id повторяются.
Далее будем использовать только уникальные id.

"""
print(len(objids))
print(len(set(objids)))

31304
32328


# Данные по определенному объекту

In [20]:
# Для скачивания данных по определенному объекту, нужно добавить его id в конец строки запроса
#url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/29679'

In [50]:
info_all_objects = []
for i in tqdm(set(objids)):
    res = requests.get(f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{i}')
    info_all_objects += [res.json()]

100%|██████████| 31304/31304 [2:06:07<00:00,  4.14it/s]   


In [57]:
all_objects_df = pd.json_normalize([i.get('data') for i in info_all_objects])
all_objects_df.head()

,id,pdId,region,address,nameObj,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,developer.orgBankruptMsgDttm,newBuildingId,conclusion,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors
0,7.0,52.0,77.0,"г Москва, район Теплый стан, ул Профсоюзная, в...","«ДОМ №128», «ДОМ 128»",21.0,21.0,0.0,2019-04-09,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.0,14.0,77.0,"д Столбово, д. 2","ЖК ""Москвичка""",2.0,15.0,1553.0,2019-12-12,Панель,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9.0,86.0,50.0,"ГОРОД БАЛАШИХА, УЛИЦА ЛУКИНО, вл. 51","ЖК ""Новая Алексеевская роща""",NaN,NaN,108.0,2018-12-14,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10.0,92.0,77.0,"п Москва, д. 2, корпус 4",NaN,NaN,NaN,1376.0,2019-03-25,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11.0,102.0,77.0,"поселение Сосенское, вблизи д. Столбово, д. 2,...",NaN,NaN,NaN,709.0,2018-12-12,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Преобразование и сохранение датафрейма

In [60]:
'''

Сохраняем датафрейм в разные форматы

'''

all_objects_df.to_pickle('data/all_objects_df.pkl')
all_objects_df.to_excel('data/all_objects_df.xlsx')

In [174]:
'''
Для сохранения ДФ в БД SQL необходимо преобразовать или исключить колонки с множественной вложенностью словарей, т.к. будет конфликт типов столбцов

Вижу 2 пути:
1) преобразовать столбцы в текстовый формат, чтобы сохранить ДФ в неизменном виде;
2) удалить колонки или оставить в них только нужную информацию.

Решил пойти по 2-му пути, т.к. есть 4 такие колонки. В одной из них содержится проектная документация, которая не понадобится
во второй и третьей содержится информация только по одному объекту из всех,
а в четвертой содержится информация о фотографиях и еще 2 нужные колонки. Эту колонку мы преобразуем.

Т.к. для загрузки в БД немного преобразовываем ДФ, то пустые строки целесообразно удалить сразу
(с сайта получен ответ об отсутсвии информации по ID с ошибкой 404).

'''

all_objects_df = pd.read_pickle('data/all_objects_df.pkl')

In [175]:
'''
Удаляем все строки, где не указан id. При загрузке данных по id по ним была получена ошибка 404.
Удаляем столбец в котором загружена информация по проектной документации,
т.к. эти данные нам не требуются, и удаляем столбцы в которых имеется запись только по одному объекту
'''
all_objects_df = all_objects_df.drop([i for i in all_objects_df[all_objects_df.id.isna()].index], axis=0)
all_objects_df = all_objects_df.drop(['rnvDTO', 'objectTransportInfo', 'metro.colors'], axis=1)
all_objects_df.shape

(30532, 107)

In [176]:
"""
Т.к. есть объекты без фото, то создадим список только с объектами, в которых есть фото.
Получилось, что всего есть 64394 фото (в некоторых объектах есть несколько фото).
"""
photo_column = []
for i in all_objects_df.photoRenderDTO:
    if i:
        photo_column += i
len(photo_column)

64394

In [177]:
"""
Создаем новый дф
"""
df_photo_column = pd.json_normalize(photo_column)

In [178]:
'''
В этом дф нас интересует только несколько колонок. Колонку с id переименовываем, чтобы она совпадала с другим дф
'''
df_photo_column = df_photo_column.rename(columns={'objId':'id'})
df_photo_column = df_photo_column[['id', 'objRenderPhotoUrl', 'objReadyDesc', 'objBuildTypeShortDesc']]
df_photo_column.head()

,id,objRenderPhotoUrl,objReadyDesc,objBuildTypeShortDesc
0,7,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Нежилое
1,7,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Нежилое
2,8,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Жилое
3,9,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Жилое
4,9,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Жилое


In [179]:
"""
Т.к. после создания ДФ есть записи, в которых нет ссылки на фото, то их удаляем

"""
df_photo_column = df_photo_column.drop([i for i in df_photo_column[df_photo_column.objRenderPhotoUrl.isna()].index], axis=0)

In [180]:
df_photo_column.shape

(64387, 4)

In [181]:
"""
Сделаем ДФ, в котором по каждому объекту имеется только одно фото (в принципе, можно было удалить),
информация о статусе готовности дома и статусе жилое\нежилое (т.к. в основном ДФ этой информации нет)
"""
df_to_merge = df_photo_column.drop_duplicates('id')
df_to_merge.head()

,id,objRenderPhotoUrl,objReadyDesc,objBuildTypeShortDesc
0,7,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Нежилое
2,8,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Жилое
3,9,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Жилое
6,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Жилое
7,11,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Жилое


In [182]:
"""

Создадим новый ДФ, который будет состоять из основного и дополнительных колонок, которые были сформированы выше

"""
merged_df = all_objects_df.merge(df_to_merge, on='id')
merged_df.head()

,id,pdId,region,address,nameObj,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,conclusion,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,objRenderPhotoUrl,objReadyDesc,objBuildTypeShortDesc
0,7.0,52.0,77.0,"г Москва, район Теплый стан, ул Профсоюзная, в...","«ДОМ №128», «ДОМ 128»",21.0,21.0,0.0,2019-04-09,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Нежилое
1,8.0,14.0,77.0,"д Столбово, д. 2","ЖК ""Москвичка""",2.0,15.0,1553.0,2019-12-12,Панель,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Жилое
2,9.0,86.0,50.0,"ГОРОД БАЛАШИХА, УЛИЦА ЛУКИНО, вл. 51","ЖК ""Новая Алексеевская роща""",NaN,NaN,108.0,2018-12-14,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Жилое
3,10.0,92.0,77.0,"п Москва, д. 2, корпус 4",NaN,NaN,NaN,1376.0,2019-03-25,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Жилое
4,11.0,102.0,77.0,"поселение Сосенское, вблизи д. Столбово, д. 2,...",NaN,NaN,NaN,709.0,2018-12-12,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,Сдан,Жилое


In [183]:
"""
Удаляем колонку из которой была извлечена необходимая информация
"""

merged_df = merged_df.drop('photoRenderDTO', axis=1)

In [184]:
"""
Сохраняем преобразованный ДФ в базу SQL
"""

conn = sqlite3.connect('data/test_database.db')
merged_df.to_sql('real_estate', conn, if_exists='replace', index=False)
conn.commit()
conn.close()

# Загрузка фото стоящихся жилых комплексов. Загрузил только первые 30 фото, но можно убрать ограничение и загрузить все

In [185]:
# отбираем строящиеся объекты
all_under_construction = df_photo_column[df_photo_column.objReadyDesc == "Строится"]
start_dir = os.getcwd()
# пробуем создать директорию data, если ее нет, если есть, то ничего не делаем
try:
    os.mkdir('data')
except FileExistsError:
    pass
# пробуем создать директорию pics в data, если ее нет, если есть, то ничего не делаем
try:
    os.mkdir('data/pics')
except FileExistsError:
    pass
# запускаем цикл по закачке фото, понадобится индекс для обращения к ячейкам, поэтому перебираем числа через range()
for i in tqdm(range(len(all_under_construction.head(30)))):
    # пробуем создать директорию с номером id, т.к. по одному id может быть несколько фото
    try:
        os.mkdir(f'data/pics/{all_under_construction.id.iloc[i]}')
    # если уже есть такая, то ничего не делаем
    except FileExistsError:
        pass
    # меняем директорию на директорию с id дома
    os.chdir(f'data/pics/{all_under_construction.id.iloc[i]}')
    # вытаскиваем из датафрейма ссылку на фото
    link = all_under_construction.objRenderPhotoUrl.iloc[i]
    # проверяем, есть ли ссылка или там внесена какая-то посторонняя запись
    if str(link).startswith('htt'):
        # скачиваем фото
        wget.download(link)
    # меняем директорию на ту, с которой начинали
    os.chdir(start_dir)

100%|██████████| 30/30 [00:22<00:00,  1.36it/s]
